In [1]:
import pandas as pd
import numpy as np
import re

from pathlib import Path

path = "../data/donnees_communales_parc_jardin.csv"

def compose_address(row):
    parts = []
    for c in ["num", "numvoie", "voie"]:
        if c in row.index and pd.notna(row[c]) and str(row[c]).strip():
            parts.append(str(row[c]).strip())
    return " ".join(parts).strip() or np.nan

# Infos

In [2]:
df = pd.read_csv(path)
display(df.head(3)) 
df.info()

,FID,uid,id_ariane,nom,num,numvoie,voie,codepost,commune,code_insee,...,eau,toilettes,chien,esp_can,photo,gid,the_geom,openinghours,last_update_fme,horaires
0,com_donnees_communales.comparcjardin_1_0_0.371,PAR-69204-001,NaN,Parc historique de Beauregard,1,NaN,Rue Edouard Millaud,69230.0,Saint-Genis-Laval,69204.0,...,oui,oui,Tenus en laisse,Non,NaN,371,"MULTIPOLYGON (((4.786552 45.69293704090367, 4....","[""Mo-Su 08:00-18:00""]",2022-03-31T14:13:25.31,NaN
1,com_donnees_communales.comparcjardin_1_0_0.372,PAR-69204-002,NaN,Parc de la Maison des Champs,2,130,Avenue Georges Clemenceau,69230.0,Saint-Genis-Laval,69204.0,...,non,non,Tenus en laisse,Propreté,NaN,372,"MULTIPOLYGON (((4.792575 45.69654604090331, 4....","[""Mo-Su 08:00-18:00""]",2022-03-31T14:13:25.63,NaN
2,com_donnees_communales.comparcjardin_1_0_0.76,PAR-69385-05110,S8220,Jardin André Malraux,5110,NaN,Place des Minimes,69005.0,Lyon 5e Arrondissement,69385.0,...,oui,non,Oui,Non,0,76,MULTIPOLYGON (((4.821792696554102 45.758162063...,[],2025-07-25T08:55:31.762,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   FID                        627 non-null    object 
 1   uid                        627 non-null    object 
 2   id_ariane                  318 non-null    object 
 3   nom                        627 non-null    object 
 4   num                        627 non-null    int64  
 5   numvoie                    324 non-null    object 
 6   voie                       623 non-null    object 
 7   codepost                   625 non-null    float64
 8   commune                    625 non-null    object 
 9   code_insee                 625 non-null    float64
 10  reglement                  568 non-null    object 
 11  surf_tot_m2                577 non-null    float64
 12  gestion                    621 non-null    object 
 13  ann_ouvert                 120 non-null    float64

# Standardisation des noms et nettoyage de base

In [3]:
parcs = pd.DataFrame()
parcs["adresse"] = df.apply(compose_address, axis=1)
parcs["commune"] = df["commune"].astype(str).str.strip()
parcs["surface_m2"] = (
    df["surf_tot_m2"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .str.extract(r"([\d\.]+)", expand=False)
    .astype(float)
)

if "type_equip" in df.columns:
    parcs["aire_jeu"] = df["type_equip"].str.contains("Aire de jeux", case=False, na=False)
else:
    parcs["aire_jeu"] = False

if "eau" in df.columns:
    parcs["eau"] = df["eau"].astype(str).str.lower().eq("oui")
else:
    parcs["eau"] = False

if "toilettes" in df.columns:
    parcs["toilettes"] = df["toilettes"].astype(str).str.lower().eq("oui")
else:
    parcs["toilettes"] = False


# Suppression des colonnes inutiles

In [4]:
colonnes_a_supprimer = [
    "FID","uid","id_ariane","codepost","esp_can","photo","openinghours",
    "last_update_fme","horaires","reglement","gestion","ann_ouvert",
    "precision_horaires","circulation","acces","label","clos",
    "openinghoursspecification","gid"
]
df = df.drop(columns=[c for c in colonnes_a_supprimer if c in df.columns], errors="ignore")


# Nettoyage du contenu

In [5]:
if "code_insee" in df.columns:
    df["code_insee"] = df["code_insee"].astype(str).str.replace(r"\.0$", "", regex=True).str.strip()
    parcs["code_insee"] = df["code_insee"]

if "the_geom" in df.columns:
    def extraire_coords(geom):
        if pd.isna(geom): return np.nan, np.nan
        geom_str = re.sub(r"[A-Z\(\)]", " ", str(geom))
        match = re.search(r"([-+]?\d+\.\d+)\s+([-+]?\d+\.\d+)", geom_str)
        return (float(match.group(1)), float(match.group(2))) if match else (np.nan, np.nan)
    coords = df["the_geom"].apply(lambda g: pd.Series(extraire_coords(g)))
    coords.columns = ["longitude", "latitude"]
    parcs = pd.concat([parcs.reset_index(drop=True), coords.reset_index(drop=True)], axis=1)

def normalize_text(s):
    if pd.isna(s): return s
    return re.sub(r"\s+", " ", str(s).strip())

for col in ["adresse","commune","eau","toilettes","aire_jeu"]:
    if col in parcs.columns:
        parcs[col] = parcs[col].map(normalize_text)

parcs = parcs.dropna(subset=["adresse"])
parcs["adresse"] = parcs["adresse"].str.replace(r"\s+"," ",regex=True).str.strip()
subset_dups = [c for c in ["adresse","commune"] if c in parcs.columns]
parcs = parcs.drop_duplicates(subset=subset_dups)

In [6]:
parcs.head(10)


,adresse,commune,surface_m2,aire_jeu,eau,toilettes,code_insee,longitude,latitude
0,1 Rue Edouard Millaud,Saint-Genis-Laval,36358.950000,False,True,True,69204,4.786552,45.692937
1,2 130 Avenue Georges Clemenceau,Saint-Genis-Laval,4506.040000,False,False,False,69204,4.792575,45.696546
2,5110 Place des Minimes,Lyon 5e Arrondissement,3334.820000,False,True,False,69385,4.821793,45.758162
3,6073 Rue Tête d'Or,Lyon 6e Arrondissement,2094.410000,True,True,False,69386,4.852025,45.766371
4,8138 7 Rue du Presbytere,Lyon 8e Arrondissement,1330.870000,True,False,False,69388,4.851964,45.735282
5,7105 390 Rue Garibaldi,Lyon 7e Arrondissement,2037.580000,True,False,False,69387,4.846837,45.744334
6,6049 147 Rue de Créqui,Lyon 6e Arrondissement,2642.520000,False,False,False,69386,4.847663,45.764203
7,8 10 Boulevard De Lattre de Tassigny,Rillieux-la-Pape,10804.000000,True,True,True,69286,4.904408,45.812028
8,56130 2 Rue Barème,Lyon 6e Arrondissement,NaN,False,False,False,69386,4.842589,45.774183
9,40 33 Rue Armand,Villeurbanne,3423.625396,True,True,False,69266,4.886612,45.779803


# export des donner

In [7]:
parcs.to_csv("../csv_clean/parcs_jardins_communes_clean.csv", index=False)
print("✅ Fichier écrit : parcs_jardins_communes_agg.csv")

✅ Fichier écrit : parcs_jardins_communes_agg.csv
